## Kaggle – DataTops®
Luismi ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a Luismi a mejorar ese modelo?

## Métrica: 
Error de raíz cuadrada media (RMSE) es la desviación estándar de los valores residuales (errores de predicción). Los valores residuales son una medida de la distancia de los puntos de datos de la línea de regresión; RMSE es una medida de cuál es el nivel de dispersión de estos valores residuales. En otras palabras, le indica el nivel de concentración de los datos en la línea de mejor ajuste.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## Librerías

In [19]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import urllib.request
import bootcampviztools as bvz
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split, GridSearchCV




## Datos

In [20]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df_train = pd.read_csv("./data/train.csv", index_col = 0)
df_train.index.name = None
df_train

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00
...,...,...,...,...,...,...,...,...,...,...,...,...
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00


## Exploración de los datos

In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB


## Procesado de datos

Nuestro target es la columna `Price_in_euros`

In [22]:
import pandas as pd
import re

# Copia del dataframe original para transformaciones
df_train_processed = df_train.copy()

# 1) Mantener Company sin transformar (CatBoost maneja categóricas)

# 2) Transformar Product (primera palabra o última si es alfanumérico)
def extract_product_keyword(product):
    words = product.split()
    if words[-1].isalnum() and any(c.isalpha() for c in words[-1]) and any(c.isdigit() for c in words[-1]):
        return words[-1]  # Si es alfanumérico, tomar la última palabra
    return words[0]  # En otro caso, tomar la primera palabra

df_train_processed["Product"] = df_train_processed["Product"].apply(extract_product_keyword)

# 3) Mantener TypeName sin transformar

# 4) Transformar ScreenResolution tomando solo el último código alfanumérico
def extract_screen_res(resolution):
    matches = re.findall(r'\d+x\d+', resolution)
    return matches[-1] if matches else "Unknown"

df_train_processed["ScreenResolution"] = df_train_processed["ScreenResolution"].apply(extract_screen_res)

# 5) Dividir CPU en tres columnas
def extract_cpu_parts(cpu):
    parts = cpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    third_word = parts[2] if len(parts) > 2 else "Unknown"
    last_word = parts[-1] if len(parts) > 0 else "Unknown"
    return first_word, third_word, last_word

df_train_processed[["CPU_Brand", "CPU_Model", "CPU_Speed"]] = df_train_processed["Cpu"].apply(lambda x: pd.Series(extract_cpu_parts(x)))

# Transformar CPU_Model en una variable numérica con sentido
def categorize_cpu(cpu):
    if "i7" in cpu:
        return 3
    elif "i5" in cpu:
        return 2
    elif "i3" in cpu:
        return 1
    else:
        return 0

df_train_processed["CPU_Model_number"] = df_train_processed["CPU_Model"].apply(categorize_cpu)

# 6) Dividir Memory en dos columnas
def extract_memory_parts(memory):
    parts = memory.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    last_word = parts[-1] if len(parts) > 1 else "Unknown"
    return first_word, last_word

df_train_processed[["Memory_Capacity", "Memory_Type"]] = df_train_processed["Memory"].apply(lambda x: pd.Series(extract_memory_parts(x)))

# 7) Dividir Gpu en tres columnas
def extract_gpu_parts(gpu):
    parts = gpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    second_word = parts[1] if len(parts) > 1 else "Unknown"
    last_word = parts[-1] if len(parts) > 2 else "Unknown"
    return first_word, second_word, last_word

df_train_processed[["GPU_Brand", "GPU_Type", "GPU_Model"]] = df_train_processed["Gpu"].apply(lambda x: pd.Series(extract_gpu_parts(x)))

# 8) Reducir OpSys a seis categorías
def categorize_os(os):
    os = os.lower()
    if "windows" in os:
        return "Windows"
    elif "linux" in os:
        return "Linux"
    elif "mac" in os or "macos" in os:
        return "Mac"
    elif "android" in os:
        return "Android"
    elif "chrome" in os or "chromebook" in os:
        return "Chrome"
    else:
        return "Others"

df_train_processed["OpSys_type"] = df_train_processed["OpSys"].apply(categorize_os)
# Extraer el número del sistema operativo si existe
df_train_processed["OpSys_number"] = df_train_processed["OpSys"].str.extract(r'(\d+)')

# Convertir a número (NaN se llenará con 0 o algún otro valor si es necesario)
df_train_processed["OpSys_number"] = df_train_processed["OpSys_number"].fillna(0).astype(int)

# Limpiar y convertir columnas numéricas

# 1) Convertir Ram a numérico (eliminando "GB")
df_train_processed["Ram"] = df_train_processed["Ram"].str.replace("GB", "", regex=True).astype(int)

# 2) Convertir Weight a numérico (eliminando "kg")
df_train_processed["Weight"] = df_train_processed["Weight"].str.replace("kg", "", regex=True).astype(float)

# 3) Crear nuevas características
# Calcular el número total de píxeles
df_train_processed["Pixel_Count"] = df_train_processed["ScreenResolution"].apply(lambda x: int(x.split('x')[0]) * int(x.split('x')[1]) if 'x' in x else 0)

# Densidad de píxeles por pulgada (DPI)
df_train_processed["DPI"] = df_train_processed["Pixel_Count"] / (df_train_processed["Inches"] ** 2)

# Relación entre peso y tamaño de pantalla
df_train_processed["Weight_per_Inch"] = df_train_processed["Weight"] / df_train_processed["Inches"]

# Relación CPU y RAM
df_train_processed["CPU_Ratio"] = df_train_processed["CPU_Brand"].astype(str) + "_" + df_train_processed["Ram"].astype(str)

# Mantener las columnas categóricas como strings para CatBoost
df_train_processed["Company"] = df_train_processed["Company"].astype(str)
df_train_processed["Product"] = df_train_processed["Product"].astype(str)
df_train_processed["TypeName"] = df_train_processed["TypeName"].astype(str)
df_train_processed["ScreenResolution"] = df_train_processed["ScreenResolution"].astype(str)
df_train_processed["CPU_Brand"] = df_train_processed["CPU_Brand"].astype(str)
df_train_processed["CPU_Model"] = df_train_processed["CPU_Model"].astype(str)
df_train_processed["CPU_Speed"] = df_train_processed["CPU_Speed"].str[:-3].astype(float)
df_train_processed["Memory_Capacity"] = df_train_processed["Memory_Capacity"].str.replace('TB', '000').str.replace('GB', '').astype(float)
df_train_processed["Memory_Type"] = df_train_processed["Memory_Type"].astype(str)
df_train_processed["GPU_Brand"] = df_train_processed["GPU_Brand"].astype(str)
df_train_processed["GPU_Model"] = df_train_processed["GPU_Model"].astype(str)
df_train_processed["OpSys"] = df_train_processed["OpSys"].astype(str)

# Eliminar columnas originales que ya fueron transformadas
df_train_processed.drop(columns=["OpSys", "Cpu", "Memory", "Gpu"], inplace=True)

# Renombrar el dataset final
df_train_processed = df_train_processed

df_train_processed



,Company,Product,TypeName,Inches,ScreenResolution,Ram,Weight,Price_in_euros,CPU_Brand,CPU_Model,...,Memory_Type,GPU_Brand,GPU_Type,GPU_Model,OpSys_type,OpSys_number,Pixel_Count,DPI,Weight_per_Inch,CPU_Ratio
755,HP,G6,Notebook,15.6,1920x1080,8,1.86,539.00,Intel,i3,...,SSD,Intel,HD,520,Windows,10,2073600,8520.710059,0.119231,Intel_8
618,Dell,Inspiron,Gaming,15.6,1920x1080,16,2.59,879.01,Intel,i7,...,HDD,Nvidia,GeForce,960<U+039C>,Windows,10,2073600,8520.710059,0.166026,Intel_16
909,HP,ProBook,Notebook,15.6,1920x1080,8,2.04,900.00,Intel,i7,...,HDD,Nvidia,GeForce,930MX,Windows,10,2073600,8520.710059,0.130769,Intel_8
2,Apple,Macbook,Ultrabook,13.3,1440x900,8,1.34,898.94,Intel,i5,...,Storage,Intel,HD,6000,Mac,0,1296000,7326.587145,0.100752,Intel_8
286,Dell,Inspiron,Notebook,15.6,1920x1080,4,2.25,428.00,Intel,i3,...,HDD,AMD,Radeon,M430,Linux,0,2073600,8520.710059,0.144231,Intel_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,Dell,Inspiron,Notebook,15.6,1920x1080,8,2.20,800.00,Intel,i5,...,SSD,AMD,Radeon,530,Windows,10,2073600,8520.710059,0.141026,Intel_8
1160,HP,Spectre,2 in 1 Convertible,13.3,1920x1080,8,1.48,1629.00,Intel,i5,...,SSD,Intel,HD,520,Windows,10,2073600,11722.539431,0.111278,Intel_8
78,Lenovo,IdeaPad,Notebook,15.6,1920x1080,8,2.20,519.00,Intel,i5,...,HDD,Intel,HD,620,Others,0,2073600,8520.710059,0.141026,Intel_8
23,HP,G6,Notebook,15.6,1366x768,4,1.86,258.00,AMD,E2-9000e,...,HDD,AMD,Radeon,R2,Others,0,1049088,4310.848126,0.119231,AMD_4


In [23]:
df_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Ram               912 non-null    int32  
 6   Weight            912 non-null    float64
 7   Price_in_euros    912 non-null    float64
 8   CPU_Brand         912 non-null    object 
 9   CPU_Model         912 non-null    object 
 10  CPU_Speed         912 non-null    float64
 11  CPU_Model_number  912 non-null    int64  
 12  Memory_Capacity   912 non-null    float64
 13  Memory_Type       912 non-null    object 
 14  GPU_Brand         912 non-null    object 
 15  GPU_Type          912 non-null    object 
 16  GPU_Model         912 non-null    object 
 17  

-----------------------------------------------------------------------------------------------------------------

In [24]:
df_test = pd.read_csv("./data/test.csv", index_col=0)
df_test.index.name = None
df_test

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg
...,...,...,...,...,...,...,...,...,...,...,...
820,MSI,GE72MVR 7RG,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
948,Toshiba,Tecra Z40-C-12X,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.47kg
483,Dell,Precision M5520,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg
1017,HP,Probook 440,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.64kg


 ## 2. Replicar el procesado para ``test.csv``

In [25]:
# Copia del dataframe original para transformaciones
df_test_processed = df_test.copy()

# 1) Mantener Company sin transformar (CatBoost maneja categóricas)

# 2) Transformar Product (primera palabra o última si es alfanumérico)
def extract_product_keyword(product):
    words = product.split()
    if words[-1].isalnum() and any(c.isalpha() for c in words[-1]) and any(c.isdigit() for c in words[-1]):
        return words[-1]  # Si es alfanumérico, tomar la última palabra
    return words[0]  # En otro caso, tomar la primera palabra

df_test_processed["Product"] = df_test_processed["Product"].apply(extract_product_keyword)

# 3) Mantener TypeName sin transformar

# 4) Transformar ScreenResolution tomando solo el último código alfanumérico
def extract_screen_res(resolution):
    matches = re.findall(r'\d+x\d+', resolution)
    return matches[-1] if matches else "Unknown"

df_test_processed["ScreenResolution"] = df_test_processed["ScreenResolution"].apply(extract_screen_res)

# 5) Dividir CPU en tres columnas
def extract_cpu_parts(cpu):
    parts = cpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    third_word = parts[2] if len(parts) > 2 else "Unknown"
    last_word = parts[-1] if len(parts) > 0 else "Unknown"
    return first_word, third_word, last_word

df_test_processed[["CPU_Brand", "CPU_Model", "CPU_Speed"]] = df_test_processed["Cpu"].apply(lambda x: pd.Series(extract_cpu_parts(x)))

# Transformar CPU_Model en una variable numérica con sentido
df_test_processed["CPU_Model_number"] = df_test_processed["CPU_Model"].apply(categorize_cpu)

# 6) Dividir Memory en dos columnas
def extract_memory_parts(memory):
    parts = memory.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    last_word = parts[-1] if len(parts) > 1 else "Unknown"
    return first_word, last_word

df_test_processed[["Memory_Capacity", "Memory_Type"]] = df_test_processed["Memory"].apply(lambda x: pd.Series(extract_memory_parts(x)))

# 7) Dividir Gpu en tres columnas
def extract_gpu_parts(gpu):
    parts = gpu.split()
    first_word = parts[0] if len(parts) > 0 else "Unknown"
    second_word = parts[1] if len(parts) > 1 else "Unknown"
    last_word = parts[-1] if len(parts) > 2 else "Unknown"
    return first_word, second_word, last_word
df_test_processed[["GPU_Brand", "GPU_Type", "GPU_Model"]] = df_test_processed["Gpu"].apply(lambda x: pd.Series(extract_gpu_parts(x)))

# 8) Reducir OpSys a seis categorías
def categorize_os(os):
    os = os.lower()
    if "windows" in os:
        return "Windows"
    elif "linux" in os:
        return "Linux"
    elif "mac" in os or "macos" in os:
        return "Mac"
    elif "android" in os:
        return "Android"
    elif "chrome" in os or "chromebook" in os:
        return "Chrome"
    else:
        return "Others"

df_test_processed["OpSys_type"] = df_test_processed["OpSys"].apply(categorize_os)
# Extraer el número del sistema operativo si existe
df_test_processed["OpSys_number"] = df_test_processed["OpSys"].str.extract(r'(\d+)')

# Convertir a número (NaN se llenará con 0 o algún otro valor si es necesario)
df_test_processed["OpSys_number"] = df_test_processed["OpSys_number"].fillna(0).astype(int)

# Limpiar y convertir columnas numéricas

# 1) Convertir Ram a numérico (eliminando "GB")
df_test_processed["Ram"] = df_test_processed["Ram"].str.replace("GB", "", regex=True).astype(int)

# 2) Convertir Weight a numérico (eliminando "kg")
df_test_processed["Weight"] = df_test_processed["Weight"].str.replace("kg", "", regex=True).astype(float)

# 3) Crear nuevas características
# Calcular el número total de píxeles
df_test_processed["Pixel_Count"] = df_test_processed["ScreenResolution"].apply(lambda x: int(x.split('x')[0]) * int(x.split('x')[1]) if 'x' in x else 0)

# Densidad de píxeles por pulgada (DPI)
df_test_processed["DPI"] = df_test_processed["Pixel_Count"] / (df_test_processed["Inches"] ** 2)

# Relación entre peso y tamaño de pantalla
df_test_processed["Weight_per_Inch"] = df_test_processed["Weight"] / df_test_processed["Inches"]

# Relación CPU y RAM
df_test_processed["CPU_Ratio"] = df_test_processed["CPU_Brand"].astype(str) + "_" + df_test_processed["Ram"].astype(str)

# Mantener las columnas categóricas como strings para CatBoost
df_test_processed["Company"] = df_test_processed["Company"].astype(str)
df_test_processed["Product"] = df_test_processed["Product"].astype(str)
df_test_processed["TypeName"] = df_test_processed["TypeName"].astype(str)
df_test_processed["ScreenResolution"] = df_test_processed["ScreenResolution"].astype(str)
df_test_processed["CPU_Brand"] = df_test_processed["CPU_Brand"].astype(str)
df_test_processed["CPU_Model"] = df_test_processed["CPU_Model"].astype(str)
df_test_processed["CPU_Speed"] = df_test_processed["CPU_Speed"].str[:-3].astype(float)
df_test_processed["Memory_Capacity"] = df_test_processed["Memory_Capacity"].str.replace('TB', '000').str.replace('GB', '').astype(float)
df_test_processed["Memory_Type"] = df_test_processed["Memory_Type"].astype(str)
df_test_processed["GPU_Brand"] = df_test_processed["GPU_Brand"].astype(str)
df_test_processed["GPU_Model"] = df_test_processed["GPU_Model"].astype(str)
df_test_processed["OpSys"] = df_test_processed["OpSys"].astype(str)

# Eliminar columnas originales que ya fueron transformadas
df_test_processed.drop(columns=["OpSys", "Cpu", "Memory", "Gpu"], inplace=True)

# Renombrar el dataset final
df_test_processed = df_test_processed

df_test_processed



,Company,Product,TypeName,Inches,ScreenResolution,Ram,Weight,CPU_Brand,CPU_Model,CPU_Speed,...,Memory_Type,GPU_Brand,GPU_Type,GPU_Model,OpSys_type,OpSys_number,Pixel_Count,DPI,Weight_per_Inch,CPU_Ratio
209,Lenovo,Legion,Gaming,15.6,1920x1080,16,2.400,Intel,i7,2.8,...,SSD,Nvidia,GeForce,1060,Others,0,2073600,8520.710059,0.153846,Intel_16
1281,Acer,Aspire,Notebook,15.6,1366x768,4,2.400,Intel,Dual,1.6,...,HDD,Intel,HD,400,Linux,0,1049088,4310.848126,0.153846,Intel_4
1168,Lenovo,V110-15ISK,Notebook,15.6,1366x768,4,1.900,Intel,i3,2.0,...,HDD,Intel,HD,520,Others,0,1049088,4310.848126,0.121795,Intel_4
1231,Dell,Inspiron,2 in 1 Convertible,15.6,1920x1080,8,2.191,Intel,i5,2.5,...,SSD,Intel,HD,620,Windows,10,2073600,8520.710059,0.140449,Intel_8
1020,HP,ProBook,Notebook,14.0,1920x1080,4,1.950,Intel,i5,2.5,...,SSD,Intel,HD,620,Windows,10,2073600,10579.591837,0.139286,Intel_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,MSI,7RG,Gaming,17.3,1920x1080,16,2.900,Intel,i7,2.8,...,HDD,Nvidia,GeForce,1070,Windows,10,2073600,6928.397207,0.167630,Intel_16
948,Toshiba,Tecra,Notebook,14.0,1920x1080,4,1.470,Intel,i5,2.3,...,SSD,Intel,HD,520,Windows,10,2073600,10579.591837,0.105000,Intel_4
483,Dell,M5520,Workstation,15.6,1920x1080,8,1.780,Intel,i7,2.8,...,SSD,Nvidia,Quadro,M1200,Windows,10,2073600,8520.710059,0.114103,Intel_8
1017,HP,Probook,Notebook,14.0,1366x768,4,1.640,Intel,i5,2.5,...,HDD,Intel,HD,620,Windows,10,1049088,5352.489796,0.117143,Intel_4


In [26]:
df_test_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   Inches            391 non-null    float64
 4   ScreenResolution  391 non-null    object 
 5   Ram               391 non-null    int32  
 6   Weight            391 non-null    float64
 7   CPU_Brand         391 non-null    object 
 8   CPU_Model         391 non-null    object 
 9   CPU_Speed         391 non-null    float64
 10  CPU_Model_number  391 non-null    int64  
 11  Memory_Capacity   391 non-null    float64
 12  Memory_Type       391 non-null    object 
 13  GPU_Brand         391 non-null    object 
 14  GPU_Type          391 non-null    object 
 15  GPU_Model         391 non-null    object 
 16  OpSys_type        391 non-null    object 
 17  

## Modelado

In [27]:

# Separar variables predictoras y objetivo
X = df_train_processed.drop(columns=["Price_in_euros"])  # Eliminar la variable objetivo
y = df_train_processed["Price_in_euros"]

# Dividir en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar columnas categóricas (CatBoost maneja estas columnas directamente)
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

# Crear los conjuntos de datos CatBoost (maneja directamente categóricas)
train_pool = Pool(X_train, label=y_train, cat_features=categorical_features)
val_pool = Pool(X_val, label=y_val, cat_features=categorical_features)

# Definir los hiperparámetros a optimizar
param_dist = {
    "depth": [6, 8, 10, 12],
    "learning_rate": [0.005, 0.01, 0.05, 0.1],
    "iterations": [500, 1000, 1500, 2000]
}

# Inicializar el modelo de CatBoost
cat_model = CatBoostRegressor(loss_function="RMSE", verbose=0, random_state=42)

# Optimización de hiperparámetros con RandomizedSearchCV
random_search = RandomizedSearchCV(cat_model, param_distributions=param_dist, 
                                   n_iter=15, cv=3, scoring="neg_root_mean_squared_error", verbose=1, n_jobs=-1)

random_search.fit(X_train, y_train, cat_features=categorical_features)

# Obtener los mejores hiperparámetros
best_params = random_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Reentrenar el modelo con los mejores hiperparámetros
best_cat_model = CatBoostRegressor(**best_params, loss_function="RMSE", verbose=100, random_state=42)
best_cat_model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=100)

# Hacer predicciones en validación
y_pred = best_cat_model.predict(X_val)

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE en validación: {rmse}")

# Generar predicciones para Kaggle
X_test = df_test_processed  # Usar el dataset de prueba ya transformado
test_pool = Pool(X_test, cat_features=categorical_features)
y_test_pred = best_cat_model.predict(test_pool)

# Guardar las predicciones en un archivo CSV para Kaggle
df_submission = pd.DataFrame({"laptop_ID": df_test.index, "Price_in_euros": y_test_pred})
df_submission.to_csv("submission_catboost_v2.csv", index=False)

print("Predicciones guardadas en 'submission_catboost.csv'. ¡Listo para subir a Kaggle!")

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Mejores hiperparámetros: {'learning_rate': 0.1, 'iterations': 1000, 'depth': 6}
0:	learn: 636.0203294	test: 711.1480483	best: 711.1480483 (0)	total: 30.2ms	remaining: 30.1s
100:	learn: 194.6305487	test: 313.0568967	best: 313.0568967 (100)	total: 3.45s	remaining: 30.7s
200:	learn: 146.2199914	test: 300.9241448	best: 300.8671426 (198)	total: 7.21s	remaining: 28.7s
300:	learn: 120.2712603	test: 296.0672350	best: 296.0370531 (288)	total: 11s	remaining: 25.6s
400:	learn: 99.6423792	test: 295.3276135	best: 294.1697278 (363)	total: 14.5s	remaining: 21.7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 294.1697278
bestIteration = 363

Shrink model to first 364 iterations.
RMSE en validación: 294.16973151420416
Predicciones guardadas en 'submission_catboost.csv'. ¡Listo para subir a Kaggle!


### 4. Sacar métricas, valorar los modelos 

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

------------------------------------------------------------

## Optimizacion

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 1. Carga los datos de `test.csv` para predecir.


### 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [28]:
sample = pd.read_csv("./data/sample_submission.csv")
sample

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6
...,...,...
386,820,474.3
387,948,1468.8
388,483,520.4
389,1017,515.1


### 4. Mete tus predicciones en un dataframe llamado ``submission``.

### 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [29]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")